# Create Embeddings out of an LLM

### Problem Statement
Your mission, should you choose to accept it, is to extract **meaningful sentence-level embeddings** using a pre-trained **causal language model (SmolLM2-135M)** on Amazon Reviews.

You're working with a **generative language model**, but you’re not here to generate Shakespeare. Instead, you’ll tap into its **hidden states** to get semantic embeddings that capture the essence of a review — the good, the bad, and the brutally honest.

---

### Requirements

1. **Load and Tokenize Text**
   - Use the `McAuley-Lab/Amazon-Reviews-2023` dataset (subset: `raw_review_All_Beauty`).
   - Load ~10 sample reviews for testing.
   - Tokenize them using `"HuggingFaceTB/SmolLM2-135M"` tokenizer.

2. **Extract Embeddings**
   - Run the tokenized batch through the model with `output_hidden_states=True`.
   - Access the **last hidden layer** from `outputs.hidden_states[-1]`.

3. **Compute Sentence Embeddings**
   - Options:
     - If the model uses a classification token (e.g., `[CLS]`), extract its embedding.
     - For causal models (which typically don’t), **average the token embeddings** from the final layer, **excluding padding tokens**.

4. **Display the Result**
   - Show the shape of your final sentence embedding tensor.
   - Print at least one sentence-level embedding to verify everything worked.

---

### Constraints

- ❌ Do **not** use sentence-transformers or pre-built embedding tools like `bert-as-service`.
- ❌ Do **not** generate text (no `.generate()`).
- ✅ Use only Hugging Face's `AutoModelForCausalLM` and `AutoTokenizer`.
- ✅ Exclude padding tokens when computing average embeddings.
- ✅ Ensure everything runs on `cuda` if available.

---

<details>
  <summary>💡 Hint</summary>

```python
# Run model with hidden states
outputs = model(**tokenized_inputs, output_hidden_states=True, return_dict=True)

# Get the last hidden layer (batch_size, seq_len, hidden_dim)
last_hidden = outputs.hidden_states[-1]

# Use the attention mask to avoid averaging over padding
attention_mask = tokenized_inputs['attention_mask']  # (batch_size, seq_len)

# Compute masked average: zero out padding tokens
masked_embeddings = last_hidden * attention_mask.unsqueeze(-1)  # broadcast mask
summed = masked_embeddings.sum(dim=1)  # sum across tokens
count = attention_mask.sum(dim=1, keepdim=True)  # count of non-padding tokens

# Final sentence-level embeddings
sentence_embeddings = summed / count  # (batch_size, hidden_dim)


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
# Load Amazon Reviews dataset
from datasets import load_dataset
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)

c:\Users\chand\miniconda3\envs\torchleet\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating full split: 701528 examples [00:14, 50103.12 examples/s]


In [3]:
# Load SmolLM2-135M model and tokenizer
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceTB/SmolLM2-135M")
model = AutoModelForCausalLM.from_pretrained("HuggingFaceTB/SmolLM2-135M")
print(isinstance(model, torch.nn.Module))  # Should print: True

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

True


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb): LlamaRotaryEm

In [4]:
model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((576,), eps=1e-05)
    (rotary_emb): LlamaRotaryEm